최적의 하이퍼 파라미터를 찾기 위한 코드입니다.
원래는 정의해 놓은 함수를 불러와서 사용해야 했으나, 검증 과정에서 일부 함수를 직접 코드를 복사해와서 테스트 한 코드도 많습니다.

In [3]:
import cv2
from PIL import Image
import sys
import os
import numpy as np

current_dir = os.getcwd()
sys.path.append(current_dir)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
image_folder_path = './G1020/Images/'  

In [4]:
import pandas as pd
df = pd.read_csv('G1020.csv')
image_files = df['imageID'].tolist()  # 이미지 파일 이름이 있는 열 이름
labels = df['binaryLabels'].values  # 레이블이 있는 열 이름

# 이미지 크기 설정
target_size = (28, 28)

# cv2로 이미지 읽고 전처리
def load_and_extract_hue(image_path, target_size):
    image = cv2.imread(image_path)  # 이미지 읽기 (기본 BGR 형식)
    if image is None:
        print(f"Image not found: {image_path}")
        return None
    image = cv2.resize(image, target_size)  # 이미지 크기 조정
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # BGR을 HSV로 변환
    hue_channel = hsv_image[:, :, 0]  # Hue 채널만 추출
    hue_channel = hue_channel / 180.0  # Hue 값 정규화 (0~1 범위로 스케일링, Hue 범위는 0-179)
    return hue_channel

# 모든 이미지를 불러와서 리스트에 저장
images = [load_and_extract_hue(image_folder_path + img_path, target_size) for img_path in image_files]
images = np.array([img for img in images if img is not None])  # None 값 제거

# 차원 추가하여 (1020, 1, 128, 128) 형태로 변환
images = images[:, np.newaxis, :, :]

print(f"Processed Hue images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Processed Hue images shape: (1020, 1, 28, 28)
Labels shape: (1020,)


In [5]:
from mymethod.neural_network import *
from common.trainer import Trainer
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.299666680757359
=== epoch:1, train acc:0.3, test acc:0.35 ===
train loss:2.295161261354918
train loss:2.2880640116511612
train loss:2.276487901024723
train loss:2.260578256199971
train loss:2.235414098245895
train loss:2.1930039392795697
train loss:2.153535015514454
train loss:2.069305504835655
train loss:2.0053253755997895
train loss:1.9018579775991884
train loss:1.7394360760706604
train loss:1.6390295968158433
train loss:1.445256297352058
train loss:1.3545503045823604
train loss:1.1011055508901206
train loss:0.9585805251529672
train loss:0.8868065741958517
train loss:0.7706085991645261
train loss:0.7420649120967686
train loss:0.703541613771742
train loss:0.6226815200294726
train loss:0.6118299370762117
train loss:0.6065407164582828
train loss:0.5371766957795578
train loss:0.6305270337593951
train loss:0.8820547594930874
train loss:0.5089407745664396
train loss:0.5931466385162885
train loss:0.39112273042725204
train loss:0.6468999280650344
train loss:0.785948530049672
tra

In [16]:
class DeepConvNet:
    def __init__(self, input_dim=(1, 28, 28), #필터 수 조절해서 성능 조절
                 conv_param_1 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]            

In [18]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=40)

max_epochs = 20

network = DeepConvNet()
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.1926423634401764
=== epoch:1, train acc:0.71, test acc:0.69 ===
train loss:1.8036818621599462
train loss:1.8409563547465617
train loss:1.9970071750397653
train loss:1.8478479147870235
train loss:1.5068419171513934
train loss:1.4590959227558167
train loss:1.4983113626547984
train loss:1.5991708063229875
train loss:2.1358777962600706
train loss:1.4895395286648687
train loss:1.6358821248454933
train loss:1.8811459797526893
train loss:1.8741239605335551
train loss:1.5558697258941554
train loss:1.2376692131263007
train loss:2.0758130744484222
train loss:2.2944552448258713
train loss:1.498889848887047
train loss:1.703141017022388
train loss:1.9034897453749633
train loss:1.8248833220369574
train loss:1.9287133931875435
train loss:1.8595566898612774
train loss:1.6282412238864477
train loss:2.343416517445358
train loss:1.5276564859299533
train loss:1.6857717610790435
train loss:1.5764725130626596
train loss:1.7799459036732284
train loss:1.546171409477338
train loss:1.93153879877352

In [7]:
from mymethod.functions import *
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import make_classification


x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
model = LogisticRegression()
model.fit(x_train_reshaped, y_train)
y_pred = model.predict(x_test_reshaped)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.5196078431372549
혼동 행렬:
 [[84 54]
 [44 22]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.66      0.61      0.63       138
           1       0.29      0.33      0.31        66

    accuracy                           0.52       204
   macro avg       0.47      0.47      0.47       204
weighted avg       0.54      0.52      0.53       204



In [8]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
model = LinearDiscriminantAnalysis()
model.fit(x_train_reshaped, y_train)
y_pred = model.predict(x_test_reshaped)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.5490196078431373
혼동 행렬:
 [[88 50]
 [42 24]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.68      0.64      0.66       138
           1       0.32      0.36      0.34        66

    accuracy                           0.55       204
   macro avg       0.50      0.50      0.50       204
weighted avg       0.56      0.55      0.56       204



In [9]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
model = GaussianNaiveBayes()
model.fit(x_train_reshaped, y_train)
y_pred = model.predict(x_test_reshaped)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.6764705882352942
혼동 행렬:
 [[138   0]
 [ 66   0]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.68      1.00      0.81       138
           1       0.00      0.00      0.00        66

    accuracy                           0.68       204
   macro avg       0.34      0.50      0.40       204
weighted avg       0.46      0.68      0.55       204



C:\Users\hotmi\OneDrive\Desktop\4학년 2학기\딥러닝\과제\mymethod\functions.py:87: RuntimeWarning: divide by zero encountered in log
  class_conditional = -0.5 * np.sum(np.log(2. * np.pi * self.variances[cls])) - 0.5 * np.sum(((x - self.means[cls]) ** 2) / (self.variances[cls]))
C:\Users\hotmi\OneDrive\Desktop\4학년 2학기\딥러닝\과제\mymethod\functions.py:87: RuntimeWarning: invalid value encountered in divide
  class_conditional = -0.5 * np.sum(np.log(2. * np.pi * self.variances[cls])) - 0.5 * np.sum(((x - self.means[cls]) ** 2) / (self.variances[cls]))
C:\Users\hotmi\OneDrive\Desktop\4학년 2학기\딥러닝\과제\mymethod\functions.py:87: RuntimeWarning: divide by zero encountered in divide
  class_conditional = -0.5 * np.sum(np.log(2. * np.pi * self.variances[cls])) - 0.5 * np.sum(((x - self.means[cls]) ** 2) / (self.variances[cls]))
C:\Users\hotmi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sam

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)

# Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(x_train_reshaped, y_train)
y_pred_logistic = logistic_model.predict(x_test_reshaped)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy: {logistic_accuracy:.4f}")

# Linear Discriminant Analysis (LDA) model
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(x_train_reshaped, y_train)
y_pred_lda = lda_model.predict(x_test_reshaped)
lda_accuracy = accuracy_score(y_test, y_pred_lda)
print(f"LDA Accuracy: {lda_accuracy:.4f}")

# Gaussian Naive Bayes classifier
gaussian_model = GaussianNB()
gaussian_model.fit(x_train_reshaped, y_train)
y_pred_gaussian = gaussian_model.predict(x_test_reshaped)
gaussian_accuracy = accuracy_score(y_test, y_pred_gaussian)
print(f"Gaussian Classifier Accuracy: {gaussian_accuracy:.4f}")

Logistic Regression Accuracy: 0.6961
LDA Accuracy: 0.5980
Gaussian Classifier Accuracy: 0.3725


In [20]:
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np



# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train_reshaped )
X_test = scaler.transform(x_test_reshaped)

# Build a simple TensorFlow model for binary classification
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
print(f"TensorFlow Model Accuracy: {accuracy:.4f}")

Epoch 1/20


C:\Users\hotmi\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.4832 - loss: 0.7661 - val_accuracy: 0.6033 - val_loss: 0.6933
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7212 - loss: 0.5863 - val_accuracy: 0.6087 - val_loss: 0.6702
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7457 - loss: 0.5374 - val_accuracy: 0.6359 - val_loss: 0.6966
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7763 - loss: 0.5051 - val_accuracy: 0.6250 - val_loss: 0.6872
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7669 - loss: 0.4868 - val_accuracy: 0.6685 - val_loss: 0.6810
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8036 - loss: 0.4519 - val_accuracy: 0.6413 - val_loss: 0.7029
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7977 - loss: 0.4298 - val_accuracy: 0.6576 - val_loss: 0.7347
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8201 - loss: 0.4104 - val_accuracy: 0.6359 - val_loss: 0.7586
Ep

In [21]:
# Build a deeper TensorFlow model for binary classification with CNN layers
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.Reshape((X_train.shape[1], 1)),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
print(f"TensorFlow Model Accuracy: {accuracy:.4f}")


Epoch 1/20


C:\Users\hotmi\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - accuracy: 0.7001 - loss: 0.6431 - val_accuracy: 0.6848 - val_loss: 0.6569
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.7053 - loss: 0.6178 - val_accuracy: 0.6848 - val_loss: 0.6377
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.6909 - loss: 0.5939 - val_accuracy: 0.6848 - val_loss: 0.6465
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.7117 - loss: 0.5636 - val_accuracy: 0.6630 - val_loss: 0.6736
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7465 - loss: 0.4968 - val_accuracy: 0.6848 - val_loss: 0.8285
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8227 - loss: 0.4349 - val_accuracy: 0.6141 - val_loss: 0.8646
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.8621 - loss: 0.3218 - val_accuracy: 0.6576 - val_loss: 0.9545
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9438 - loss: 0.2027 - val_accuracy: 0.5924 - val_loss: 1.

In [22]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=40)

max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.298641071418149
=== epoch:1, train acc:0.71, test acc:0.69 ===
train loss:2.2934049303714685
train loss:2.282243063616174
train loss:2.259613356627758
train loss:2.233853661981151
train loss:2.2009860553546714
train loss:2.1759477147285744
train loss:2.0433458215459535
train loss:2.015516936703876
train loss:1.9008936879723872
train loss:1.716120890933917
train loss:1.562397487422223
train loss:1.372472096284913
train loss:1.300841303911374
train loss:1.1302382074537105
train loss:0.8868701717967694
train loss:0.7398032063622868
train loss:0.5935327162326302
train loss:0.661110506768716
train loss:0.6556226353730414
train loss:0.7536658480500266
train loss:1.2373977764165622
train loss:0.6579699239721591
train loss:0.7406314113926293
train loss:0.6412668718210162
train loss:0.6788394406997923
train loss:0.6315239888181188
train loss:0.651816009968834
train loss:0.677653267772727
train loss:0.6933139494854753
train loss:0.7031473731990496
train loss:0.6595696902782204
train

In [23]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=40)

max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.2990463639710312
=== epoch:1, train acc:0.74, test acc:0.69 ===
train loss:2.2931606412915424
train loss:2.2831535488952452
train loss:2.264097885406605
train loss:2.2412893443293394
train loss:2.2070330730571204
train loss:2.1579869179786284
train loss:2.0661783195460206
train loss:1.9656802823041446
train loss:1.8513945484946401
train loss:1.6824599242419738
train loss:1.6223888143774272
train loss:1.3641721621130376
train loss:1.1314413185738554
train loss:0.8907968708746443
train loss:0.7774217483573294
train loss:0.6372293449424643
train loss:0.5559398899683597
train loss:0.8551802400748777
train loss:0.6835673883147017
train loss:1.2344437082968143
train loss:0.6968525385775584
train loss:0.5164321342520914
train loss:0.7993875383652477
train loss:0.8830566855862185
train loss:0.4091587321499824
train loss:0.7466832738609752
train loss:0.538933012067659
train loss:0.6082298174838331
train loss:0.5714990711727572
train loss:0.6349129575485944
train loss:0.522289566194

In [24]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=40)

max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.29939753712038
=== epoch:1, train acc:0.73, test acc:0.69 ===
train loss:2.2938691925828003
train loss:2.28605819805522
train loss:2.2737493479634634
train loss:2.2453058439709386
train loss:2.215235026566189
train loss:2.1644670050933064
train loss:2.117339595125234
train loss:2.0427171555840835
train loss:1.9073825831151048
train loss:1.856211316311138
train loss:1.6357375501655909
train loss:1.4782890419952968
train loss:1.2367037062501176
train loss:1.1187517224910508
train loss:1.0987064120436432
train loss:0.7632450204243462
train loss:0.5553110150845421
train loss:0.6959434439186544
train loss:0.5561921727960205
train loss:0.3070820095681229
train loss:0.9605470387185017
train loss:0.5022397283820682
train loss:0.5298032058203372
train loss:1.013526158103717
train loss:0.7408590186816693
train loss:0.5511494544140287
train loss:0.34003734282609377
train loss:0.7861658960225808
train loss:0.5144292331088963
train loss:0.5913415731927154
train loss:0.552202705882644
t

In [25]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.15, random_state=40)

max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.2995702905129334
=== epoch:1, train acc:0.73, test acc:0.68 ===
train loss:2.294557820508376
train loss:2.2845460869434575
train loss:2.2728519994385183
train loss:2.2559535094637964
train loss:2.2198222527615368
train loss:2.1871627068430155
train loss:2.1126492830075922
train loss:2.050646907537175
train loss:1.936180211797102
train loss:1.783214368020479
train loss:1.6193452161633388
train loss:1.5007299264436864
train loss:1.2721101760464424
train loss:1.0080850869340519
train loss:0.8293732626781388
train loss:0.6353125785972122
train loss:0.6240133162630641
train loss:0.5334274638184483
train loss:0.6416578689519067
train loss:0.5564138955965716
train loss:0.5735889558075218
train loss:1.2284138757674803
train loss:0.5272662745573669
train loss:0.7053030672113172
train loss:0.9260463276447559
train loss:0.6102931459809585
train loss:0.6093150455365736
train loss:0.5041067090287449
train loss:0.5088211940888798
train loss:0.664644351272147
train loss:0.532029754858526

In [26]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=40)

max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.299392328949975
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:2.2950046341805637
train loss:2.28712718526385
train loss:2.2698698388879457
train loss:2.24492553511601
train loss:2.205674646348848
train loss:2.1417169219281527
train loss:2.0749750754667917
train loss:1.968864852766784
train loss:1.8501291586411992
train loss:1.7240310869726816
train loss:1.5394126717962675
train loss:1.3955442477810587
train loss:1.1599159517179995
train loss:1.0925059455819703
train loss:0.8033004084160501
train loss:0.6285353247386251
train loss:0.8261636573645397
train loss:0.5075611525487587
train loss:0.4935680522292084
train loss:0.8634072172712489
train loss:0.6303847973753809
train loss:0.33558613175915714
train loss:0.33014419987371657
train loss:0.3396195489957733
train loss:1.045134434776887
train loss:0.4899260276109529
train loss:0.77401524409268
train loss:0.6153553066533987
train loss:0.6347854993510623
train loss:0.590666035566908
train loss:0.6789371925951329
tr

In [27]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=200, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.2944291463241457
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:2.286114191122709
train loss:2.2602273122761987
train loss:2.2286774683394883
train loss:2.1725467791755277
train loss:2.1280821619469386
train loss:2.019959157296381
train loss:2.0094505746799536
train loss:1.6790661036956211
train loss:1.5389636176583399
train loss:1.3921820523209143
train loss:1.105583998400359
train loss:0.8559330355825635
train loss:0.9455569539029837
train loss:0.855584597249518
train loss:1.1158071108773684
train loss:0.5224483027316812
train loss:0.6358725289617732
train loss:0.6372916280601062
train loss:0.5481721124321514
train loss:0.22073918376027546
train loss:0.5079224870147353
train loss:1.0867919570132982
train loss:0.6545479250156514
train loss:0.48146539988622694
train loss:0.2985185410910263
train loss:0.3751014434070157
train loss:0.6594143557406444
train loss:0.23204699259267286
train loss:0.970979294409586
train loss:0.6337548026912694
train loss:0.712647177216

In [28]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=400, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.292513318314815
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:2.2792742494364946
train loss:2.250934249313369
train loss:2.196822325673635
train loss:2.0925206092216646
train loss:1.9976171239244676
train loss:1.7731526652313225
train loss:1.5243969717187453
train loss:1.2092282930764253
train loss:1.014182654540482
train loss:0.9082855028758127
train loss:0.9461802480187387
train loss:0.39627725001126274
train loss:0.5440063476862593
train loss:0.7657904779949696
train loss:0.6810212057674485
train loss:0.6670300931989182
train loss:0.47206536489358
train loss:0.6339301663810212
train loss:0.512650615635484
train loss:0.5453063624844312
train loss:0.5563189137293926
train loss:0.5179742720283669
train loss:0.39063244077528314
train loss:0.6701979059251977
train loss:0.7191208600683839
train loss:0.5267160028865491
train loss:0.4936216573621567
train loss:0.33245561351013453
train loss:0.6170667142242425
train loss:0.7666090451389039
train loss:0.68242249977588

In [29]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=800, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.2794080069160065
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:2.2424703261298697
train loss:2.14955597367159
train loss:1.9927190711443015
train loss:1.833550754861264
train loss:1.7158363159691568
train loss:1.2208479633594413
train loss:1.0197192884251771
train loss:0.721341907432486
train loss:0.531215337389711
train loss:1.0970378201524553
train loss:0.8428356094815861
train loss:0.613973482149439
train loss:0.5216443441456508
train loss:0.6266810760879788
train loss:0.6956142047474093
train loss:0.5709550312132694
train loss:0.6049725969746842
train loss:0.8261295263418683
train loss:0.48729506491452373
train loss:0.7311326602475925
train loss:0.6121988278941356
train loss:0.6847808223680005
train loss:0.41921257393996647
train loss:0.504302606995542
train loss:0.5214033766757946
train loss:0.7347638736905588
train loss:0.9916072757896753
train loss:0.4079067475379836
train loss:0.29538909436854305
train loss:0.6062532296309794
train loss:0.75977716479099

In [30]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=10, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.300936351513168
=== epoch:1, train acc:0.29, test acc:0.31 ===
train loss:2.2991809705886346
train loss:2.297396478486835
train loss:2.295414605095741
train loss:2.293144494213417
train loss:2.2904360113182833
train loss:2.287101797267558
train loss:2.283421241958398
train loss:2.2783970400417735
train loss:2.2731574718267202
train loss:2.2678440036460086
train loss:2.259025840890818
train loss:2.2489293338014082
train loss:2.235726388030277
train loss:2.22755420709388
train loss:2.2108268792601065
train loss:2.193832297422564
train loss:2.178347838818064
train loss:2.1507280786418836
train loss:2.116697015420104
train loss:2.0673321829361133
train loss:2.06210934157894
train loss:2.0184264946067945
train loss:1.9425535203781084
train loss:1.9374556492579402
train loss:1.863719567045461
train loss:1.8047809724786035
train loss:1.7827131302957937
train loss:1.6823326416784794
train loss:1.625225360084676
train loss:1.5001965197676472
train loss:1.480952438871578
train loss:

In [31]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=50, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.3003261250057094
=== epoch:1, train acc:0.73, test acc:0.69 ===
train loss:2.296851492381972
train loss:2.2922660548594704
train loss:2.28514750307751
train loss:2.272854429852919
train loss:2.261872028297387
train loss:2.2381201860297693
train loss:2.2142352473977533
train loss:2.17211477345898
train loss:2.137161654434684
train loss:2.05971383397923
train loss:2.0077898074747584
train loss:1.9473260169696895
train loss:1.8245003286709238
train loss:1.6998798842366643
train loss:1.5987416592122032
train loss:1.449021986153584
train loss:1.324715003047123
train loss:1.1805909117231994
train loss:1.0181684675107978
train loss:0.9201447032919958
train loss:0.7931227792583831
train loss:0.6707622494440384
train loss:0.6706803315823174
train loss:0.4135033690454886
train loss:0.39740560701383426
train loss:0.1688032173887566
train loss:0.778656005311898
train loss:1.4902918304188222
train loss:0.9867459374620833
train loss:0.36676868852148853
train loss:0.3365539044252469
trai

In [35]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=1600, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6915285859618538
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6712874218279228
train loss:0.6930089718227419
train loss:0.6573711284883116
train loss:0.619618406751931
train loss:0.7291805605492454
train loss:0.7426553164854911
train loss:0.6635932375872062
train loss:0.6302734678812467
train loss:0.6767755604795154
train loss:0.5896515493065783
train loss:0.5628306969062276
train loss:0.522675610533059
train loss:0.6855303627175213
train loss:0.45903694304905535
train loss:0.3565530684145682
train loss:0.68145310027813
train loss:0.3406646027479913
train loss:0.8681427861238074
train loss:0.6424057359867591
train loss:0.6976845919709276
train loss:0.24439990616006088
train loss:0.7485616760353639
train loss:0.6022187434569496
train loss:0.4228569096567976
train loss:0.4340476547636383
train loss:0.6887205768351352
train loss:0.552364542083206
train loss:0.6088776406460253
train loss:0.4361076473214151
train loss:0.5002955057862055
train loss:0.5772268009805

In [36]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=800, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6892532351829087
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.69145826443175
train loss:0.6588352181910816
train loss:0.6628301596783277
train loss:0.7069022246976209
train loss:0.6480793516584941
train loss:0.6340707151578339
train loss:0.6737850235193757
train loss:0.7010888718823131
train loss:0.6649808207589925
train loss:0.6297914510777591
train loss:0.37211568345704105
train loss:0.6973146577415352
train loss:0.6000894242757904
train loss:0.5058184414592856
train loss:0.6086609850220278
train loss:0.1989079093079176
train loss:0.6419859935888382
train loss:0.8201784520148152
train loss:0.5179356024527799
train loss:0.3693570727993506
train loss:0.8001470086539836
train loss:0.6896811236273049
train loss:0.6935637965276749
train loss:0.4657833618609731
train loss:0.39636532828802074
train loss:0.5330529481176288
train loss:0.7556031826962211
train loss:0.6788462590367572
train loss:0.4760327640554903
train loss:0.6140806398347445
train loss:0.6149849949

In [37]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=400, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6909824160788863
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6814744653227081
train loss:0.6943661072527665
train loss:0.6545389237549544
train loss:0.6849004006511732
train loss:0.6789284124042931
train loss:0.6752038990788205
train loss:0.7134651111585124
train loss:0.5809431896150474
train loss:0.6653015955241987
train loss:0.4654802273841849
train loss:0.6067103935879833
train loss:0.7365855299405001
train loss:0.729275843462019
train loss:0.6099115760500659
train loss:0.5088340200037056
train loss:0.6862744062039976
train loss:0.6889831886610962
train loss:0.6044972757527932
train loss:0.6248856693274496
train loss:0.5470179511813014
train loss:0.522876561244608
train loss:0.6834554635549649
train loss:0.8205610308238478
train loss:0.4604783158472613
train loss:0.7472575799093989
train loss:0.6319409263968259
train loss:0.5623161842496363
train loss:0.6164063825068101
train loss:0.6779633956714761
train loss:0.48039636835310817
train loss:0.52599786431

In [38]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=200, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6923065324438722
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6900539516762788
train loss:0.6785795539552646
train loss:0.6926293538203626
train loss:0.6519385977281065
train loss:0.7110663507178523
train loss:0.6805593160904274
train loss:0.6310258241797849
train loss:0.5798678480290612
train loss:0.7548611800807905
train loss:0.6313526356054178
train loss:0.6620689252720314
train loss:0.5046001002308925
train loss:0.6748377156805425
train loss:0.433976593128408
train loss:0.5052319219820154
train loss:0.6282170490540857
train loss:0.7798666693645958
train loss:0.23647990442297345
train loss:0.9266857054516138
train loss:0.9696962389923849
train loss:0.5263165277857417
train loss:0.5043298190062286
train loss:0.5259392589529408
train loss:0.9584236162602074
train loss:0.7668224947740918
train loss:0.6146647313605811
train loss:0.7088964626818457
train loss:0.6283279879232773
train loss:0.5926747373608432
train loss:0.6712719964915854
train loss:0.5707726652

In [39]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6921748338013021
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.693109063585999
train loss:0.6920560667850038
train loss:0.6894129112988512
train loss:0.6810069370303801
train loss:0.6779076744468158
train loss:0.6864852345226911
train loss:0.6748340971213171
train loss:0.6480278622268467
train loss:0.6569427926364431
train loss:0.677917185628655
train loss:0.7716812994359413
train loss:0.6512146603201818
train loss:0.6143314566197547
train loss:0.5898373644420102
train loss:0.6843218834598428
train loss:0.5633830904606827
train loss:0.4055983290797739
train loss:0.6778291779270587
train loss:0.584002321449316
train loss:0.5874270549761963
train loss:0.49704735018831847
train loss:0.658078155064939
train loss:0.913076294776254
train loss:0.5158956863801503
train loss:0.5283407732177358
train loss:0.3838111457350678
train loss:0.4985338601298116
train loss:0.7666264903690381
train loss:0.66442003699832
train loss:0.51731422177642
train loss:0.3998135937118093
t

In [40]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 7, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6930378438128482
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6918337634970844
train loss:0.690765156017754
train loss:0.6869969365131162
train loss:0.6927076284710189
train loss:0.6992316447729181
train loss:0.6807970093359582
train loss:0.6830573590247719
train loss:0.6854768430300144
train loss:0.6834496291446024
train loss:0.6396679924498543
train loss:0.6458097208705056
train loss:0.5950228235092085
train loss:0.6233892406494023
train loss:0.576557222270312
train loss:0.6415549405363413
train loss:0.527669538557233
train loss:0.9912410535656729
train loss:0.5789541494877574
train loss:0.6120192510065499
train loss:0.5530885522050062
train loss:0.7097391293698327
train loss:0.5107824168023287
train loss:0.45958110499664667
train loss:0.5485884268129012
train loss:0.7148671575031547
train loss:0.8768498150540361
train loss:0.5591737669494017
train loss:0.5433385272623324
train loss:0.6980352195315926
train loss:0.6028730036358568
train loss:0.672220772433

In [41]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 9, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6918380304498158
=== epoch:1, train acc:0.28, test acc:0.31 ===
train loss:0.6937317236707203
train loss:0.6931829514006907
train loss:0.6930594339075553
train loss:0.6931397950555411
train loss:0.6930268298695752
train loss:0.6929864411400037
train loss:0.6922641694841714
train loss:0.6925514098439929
train loss:0.6912922735209209
train loss:0.6875516247828867
train loss:0.6846957059843235
train loss:0.6849112265770742
train loss:0.6838648931797373
train loss:0.6860769500608837
train loss:0.6813964634068741
train loss:0.6863235095627853
train loss:0.6728095591120169
train loss:0.6717742054864806
train loss:0.6330306239444109
train loss:0.6601433946925852
train loss:0.6058585200468326
train loss:0.6729817785093024
train loss:0.6032526726548615
train loss:0.7208666700261942
train loss:0.6823594138771565
train loss:0.7157479089555767
train loss:0.7624254672750462
train loss:0.6228062243487635
train loss:0.67189754784276
train loss:0.5175285118790867
train loss:0.631370701894

In [42]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 11, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6911790905644076
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6862119591150034
train loss:0.685485367391495
train loss:0.6866566568487992
train loss:0.6799386859416932
train loss:0.6153239405147104
train loss:0.6505677435662344
train loss:0.6808976735170668
train loss:0.6382625442384753
train loss:0.6718723114047063
train loss:0.6311368060888252
train loss:0.7512562774574695
train loss:0.6053444623191321
train loss:0.6270208769491107
train loss:0.7287983904069598
train loss:0.6636740860426523
train loss:0.6962658227265536
train loss:0.4197522307784426
train loss:0.415372281763074
train loss:0.44649365912308925
train loss:0.5989849528247913
train loss:0.6928272700375394
train loss:0.5251137607121489
train loss:0.47586585743311066
train loss:0.5022311386375431
train loss:0.9376745196431253
train loss:0.5278982461190089
train loss:0.6899388901309089
train loss:0.5194091865926646
train loss:0.5972306269917416
train loss:0.6013795075147363
train loss:0.6283295378

In [43]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 10, 'filter_size': 7, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6928510859104142
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6925172803634356
train loss:0.6907468827427171
train loss:0.6868809766851184
train loss:0.6838392521143701
train loss:0.6785353461274463
train loss:0.681205216746674
train loss:0.6741186135560235
train loss:0.6653251296043586
train loss:0.6109203954727258
train loss:0.677804737613181
train loss:0.6599351223474025
train loss:0.5841992096592434
train loss:0.756185082696882
train loss:0.5683976200699709
train loss:0.6862406893447898
train loss:0.621512027927779
train loss:0.46593635701721875
train loss:0.5455247971029709
train loss:0.41751109456403945
train loss:0.6039450841106018
train loss:0.2231483093208213
train loss:0.6476002128821756
train loss:0.5974805157937291
train loss:0.5371490851053946
train loss:0.5497884626728962
train loss:0.5351897078438173
train loss:0.31282555173231674
train loss:0.7212114957448446
train loss:0.6443047515016456
train loss:0.5236525563988916
train loss:0.52525725878

In [44]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 7, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6918437963654605
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.688096495473791
train loss:0.6815928026559086
train loss:0.6739298888440098
train loss:0.6913868488340634
train loss:0.6483725298130925
train loss:0.6487364854494777
train loss:0.7000266648225285
train loss:0.583001378020084
train loss:0.626131199720137
train loss:0.5444416635908292
train loss:0.617661673454801
train loss:0.6662156085830981
train loss:0.4844179653980313
train loss:0.6278261816534106
train loss:0.9701751093927333
train loss:0.5944772042941276
train loss:0.6134293852343675
train loss:0.4998105983013835
train loss:0.7072458497902258
train loss:0.5362388182140709
train loss:0.649059465150934
train loss:0.3859065428027339
train loss:0.619357723212177
train loss:0.4664577050519891
train loss:0.6095424250094357
train loss:0.4981509880102347
train loss:0.41301891306868593
train loss:0.5914301223928917
train loss:0.8384186949361455
train loss:0.8481634409805847
train loss:0.397746225436442

In [45]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 50, 'filter_size': 7, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6910262961089153
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6932117347161136
train loss:0.6927957472169799
train loss:0.6838038355492545
train loss:0.6915215229204905
train loss:0.6888102607750627
train loss:0.7081585793113065
train loss:0.6617782788384455
train loss:0.6566745924811196
train loss:0.676572951339313
train loss:0.6134418661346764
train loss:0.6993760873074464
train loss:0.5384792727416399
train loss:0.49485941453089544
train loss:0.5442097114765259
train loss:0.7097580140134762
train loss:0.401159377122483
train loss:0.5198604593699607
train loss:0.9078129903065012
train loss:0.9255405361425522
train loss:0.6107370396800779
train loss:0.6814750636124287
train loss:0.6336389244008314
train loss:0.44797153710528637
train loss:0.6957794977729261
train loss:0.554098995147964
train loss:0.4736733257453666
train loss:0.7094649761073322
train loss:0.6084251538159193
train loss:0.6381279182899864
train loss:0.5177922969049334
train loss:0.79243673890

In [46]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 70, 'filter_size': 7, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6911383436507729
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.691207470087991
train loss:0.6867724778554467
train loss:0.697156812827343
train loss:0.683690610163096
train loss:0.6848201740687052
train loss:0.6740038392403739
train loss:0.6737580115233769
train loss:0.6523746665759894
train loss:0.5941786864724034
train loss:0.48558377750749937
train loss:0.7403525481151885
train loss:0.617217212578431
train loss:0.4110856305647525
train loss:0.8267959037753829
train loss:0.5411663108564466
train loss:0.48098689801687416
train loss:0.530296148052061
train loss:0.660507217563405
train loss:0.8397987103276101
train loss:0.8101840414063897
train loss:0.9359122579167469
train loss:0.5595108298158914
train loss:0.5607255812651839
train loss:0.7129124764993157
train loss:0.6489128485522172
train loss:0.7113401364395707
train loss:0.6525790512543111
train loss:0.6360480805974185
train loss:0.6544045481247247
train loss:0.6170643733054604
train loss:0.68391717376902

In [47]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 90, 'filter_size': 7, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6924779943161419
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6926909914378856
train loss:0.6855293725598582
train loss:0.6715346159879181
train loss:0.6870807108399647
train loss:0.6799992836912911
train loss:0.6835026797128686
train loss:0.6146721804176252
train loss:0.47733728410253545
train loss:0.5366248140884652
train loss:0.6347613570054921
train loss:0.5332917612792856
train loss:0.4748212410779208
train loss:0.7113705690544088
train loss:0.9457437921535401
train loss:0.3246243326625943
train loss:0.6232017580181923
train loss:0.6214753345509948
train loss:0.8313905261094948
train loss:0.7299656958797537
train loss:0.5118277536857944
train loss:0.5155842789833246
train loss:0.6912544399953968
train loss:0.6272446658556643
train loss:0.6357370783327737
train loss:0.6302118909663544
train loss:0.6124971604271378
train loss:0.5951785901076458
train loss:0.6424014961105735
train loss:0.5966199286452678
train loss:0.6665045889607091
train loss:0.631094778

In [48]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 110, 'filter_size': 7, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6869525648098208
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6834615286589522
train loss:0.6729306193701378
train loss:0.6597401735831118
train loss:0.6389340485999555
train loss:0.6006829584520961
train loss:0.6067286135443066
train loss:0.48863436202251914
train loss:0.6602840345244683
train loss:0.6422248257610947
train loss:0.4974618129244286
train loss:0.4971678745112892
train loss:0.4914407639562469
train loss:0.810338026456086
train loss:0.4854029786436332
train loss:0.7257892211830947
train loss:0.5535576350783502
train loss:0.5417486691612836
train loss:0.7950234079051154
train loss:0.6267350912397437
train loss:0.6827541088122507
train loss:0.5912213267202008
train loss:0.595306281973968
train loss:0.5548173263684502
train loss:0.5976804623125731
train loss:0.5082244418714369
train loss:0.4973135197543835
train loss:0.6550430425238998
train loss:0.6879591490150107
train loss:0.3057740958926156
train loss:0.604443049887983
train loss:0.991569976700

In [49]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 70, 'filter_size': 7, 'pad': 1, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6911749339625759
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6903336096113669
train loss:0.6795833979723682
train loss:0.6926419595421216
train loss:0.6410342170984362
train loss:0.6169564096246152
train loss:0.673437594305627
train loss:0.5616706981997166
train loss:0.6916094283574282
train loss:0.6860281997290893
train loss:0.4207828419232099
train loss:0.626932099861168
train loss:0.8259540896017
train loss:0.4323317688561974
train loss:0.5095674807883015
train loss:0.8540933145415158
train loss:0.6059391515413177
train loss:0.4196633183608385
train loss:0.6126151586912271
train loss:0.6190351554658757
train loss:0.682590004600151
train loss:0.6264260867446012
train loss:0.7363405146595927
train loss:0.4756560052989056
train loss:0.7615529572768612
train loss:0.5576271428294965
train loss:0.5535321043806887
train loss:0.6805571902216443
train loss:0.7107271392359648
train loss:0.5047614666792459
train loss:0.5472358590001405
train loss:0.5591413238684126

In [50]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 90, 'filter_size': 7, 'pad': 2, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.690724834628417
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6813511576643965
train loss:0.6916391971363883
train loss:0.6827638528886916
train loss:0.6926381177956659
train loss:0.6541170062484747
train loss:0.642494572055041
train loss:0.5795511526229744
train loss:0.6094598725301135
train loss:0.6311812602660193
train loss:0.6402066602603147
train loss:0.49951082079951875
train loss:0.6547229844724136
train loss:0.3642587029936785
train loss:0.4703559048691076
train loss:0.5139455775923983
train loss:0.16471286982786434
train loss:0.3010474754900809
train loss:0.7101819707373213
train loss:0.8337331251287281
train loss:0.7579549962684904
train loss:0.655452271511401
train loss:0.7924620552291932
train loss:0.7375785312270187
train loss:0.6549787801761748
train loss:0.6846505957496742
train loss:0.6128711808401452
train loss:0.6673712012774222
train loss:0.6466645907658956
train loss:0.6407687554852607
train loss:0.6804002829568278
train loss:0.67756241281

In [52]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 90, 'filter_size': 7, 'pad': 2, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=20,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6876137153397166
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6912227628752132
train loss:0.6828700642460732
train loss:0.6642679949775651
train loss:0.6486703809547707
train loss:0.6687435170504311
train loss:0.6402918417060333
train loss:0.6383884123370734
train loss:0.5086445813759763
train loss:0.5287137310753158
train loss:0.6093183444210533
train loss:0.8087701538918262
train loss:0.7091399039993552
train loss:0.6496924660890607
train loss:0.6948435299658592
train loss:0.6474852820167182
train loss:0.5725119594280739
train loss:0.63301807890932
train loss:0.6788207397984704
train loss:0.6393056245670975
train loss:0.6537788164413525
train loss:0.6469591582495606
train loss:0.6187662114853665
train loss:0.6458464202296453
train loss:0.6459383025534139
train loss:0.6344724961167358
train loss:0.5951933783524972
train loss:0.6593364258080768
train loss:0.5665102927965929
train loss:0.5171616684627466
train loss:0.6898635274917723
train loss:0.687765667063

In [53]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 90, 'filter_size': 7, 'pad': 2, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=30,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6908975949154785
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6804822134633867
train loss:0.6912576687046675
train loss:0.6498729891198651
train loss:0.6564544589905207
train loss:0.5792550264734586
train loss:0.647571055260105
train loss:0.6212280116954986
train loss:0.5064970445830964
train loss:0.5873141468020416
train loss:0.5648542118503058
train loss:0.5379519474213648
train loss:0.568322518143697
train loss:0.7023306017564976
train loss:0.6534746375278221
train loss:0.6206482977299459
train loss:0.6229257343059852
train loss:0.6002851730169947
train loss:0.6443507979331751
train loss:0.6098505119278913
train loss:0.636514421079235
train loss:0.569863723119603
train loss:0.6021746159681862
train loss:0.6120112881166172
train loss:0.5335159887858033
train loss:0.6875591615366663
train loss:0.6146944606065403
train loss:0.6873682444180301
train loss:0.5157449043964344
train loss:0.558037264851179
train loss:0.5980252727643225
train loss:0.527061612257419

In [54]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 90, 'filter_size': 7, 'pad': 2, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=40,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6920259580716391
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6850742721949759
train loss:0.6756710430512316
train loss:0.6738333050009475
train loss:0.633265821688911
train loss:0.632111282281851
train loss:0.6437952087280968
train loss:0.6508215428583093
train loss:0.6449044547123213
train loss:0.5652086390602428
train loss:0.5881463884658892
train loss:0.6214469009075817
train loss:0.72938118632608
train loss:0.6554481901378093
train loss:0.5291937833280212
train loss:0.5966621600933955
train loss:0.6432911438454855
train loss:0.618392097900574
train loss:0.6626039953202938
train loss:0.5662300384307233
train loss:0.6585118006189637
train loss:0.583396869222167
train loss:0.5488877779082136
train loss:0.547093966613876
train loss:0.5021096524363559
train loss:0.528456205728997
train loss:0.5581901750365185
train loss:0.6557462567924219
train loss:0.4910582539621938
train loss:0.5913588331515325
train loss:0.6984977126650251
train loss:0.4724306551631927
t

In [55]:
network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 90, 'filter_size': 7, 'pad': 2, 'stride': 1},
                        hidden_size=100, output_size=2, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=50,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:0.6890298638239041
=== epoch:1, train acc:0.72, test acc:0.69 ===
train loss:0.6818436123032023
train loss:0.687442309088182
train loss:0.6378299325596456
train loss:0.6191798902766569
train loss:0.5989097442583547
train loss:0.6663217866518888
train loss:0.627200812326288
train loss:0.6561204118852924
train loss:0.43844264565803387
train loss:0.58997557546556
train loss:0.5290666576254316
train loss:0.49962655647046217
train loss:0.5935340911407487
train loss:0.5292443071079856
train loss:0.6340000210650631
train loss:0.58717179455823
train loss:0.6450234029028314
train loss:0.5458536703708351
train loss:0.6995484326139138
train loss:0.6634681402508968
train loss:0.699664246423847
train loss:0.6234475191805268
train loss:0.6268275651278657
train loss:0.645172314831444
train loss:0.637964011251021
train loss:0.6039102091638939
train loss:0.608066543373919
train loss:0.4943499327090054
train loss:0.5406971445084189
train loss:0.692542336392918
train loss:0.5453812559924297
tr